In [ ]:
! pip install transformers

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 


- Цель: обучить модель классифицировать комментарии на позитивные и негативные. 
    - Задача: построить модель со значением метрики качества F1 не меньше 0.75. 

## Импорты и константы

In [1]:
import torch
import torch.nn as nn
import pandas as pd
from tqdm.notebook import tqdm
import os
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import f1_score
import random

RANDOM_STATE = 1220
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 4

In [73]:
data_path_local = './toxic_comments.csv'
data_path_web = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

if os.path.exists(data_path_local):
    df = pd.read_csv(data_path_local, index_col=[0], parse_dates=[0])
elif requests.head(data_path_web).status_code == 200:
    df = pd.read_csv(data_path_web, index_col=[0], parse_dates=[0])
else:
    print('Somthing is wrong')

C:\Users\Laba-ml\AppData\Local\Temp\ipykernel_18496\448593302.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(data_path_local, index_col=[0], parse_dates=[0])
C:\Users\Laba-ml\AppData\Local\Temp\ipykernel_18496\448593302.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(data_path_local, index_col=[0], parse_dates=[0])


## Обзор данных

In [74]:
def info_df(df):
    print('------------------------------')
    print('| Информация о наборе данных |')
    print('------------------------------')
    df.info()
    print('------------------------------')
    print('| Первые 10 строчек датасета |')
    print('------------------------------')
    print(df.head(10))
    print('---------------------------------')
    print('| Последние 10 строчек датасета |')
    print('---------------------------------')
    print(df.tail(10))
    print('---------------------------')
    print('| Описательная статистика |')
    print('---------------------------')
    print(df.describe())
    print('--------------------')
    print('| Сумма дубликатов |')
    print('--------------------')
    print(df.duplicated().sum())
    print('--------------------------')
    print('| Ковариационная матрица |')
    print('--------------------------')
    plt.show()

In [75]:
info_df(df)

------------------------------
| Информация о наборе данных |
------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
------------------------------
| Первые 10 строчек датасета |
------------------------------
                                                text  toxic
0  Explanation\nWhy the edits made under my usern...      0
1  D'aww! He matches this background colour I'm s...      0
2  Hey man, I'm really not trying to edit war. It...      0
3  "\nMore\nI can't make any real suggestions on ...      0
4  You, sir, are my hero. Any chance you remember...      0
5  "\n\nCongratulations from me as well, use the ...      0
6       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK      1
7  Your vandalism to the Matt S

In [76]:
df['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

**Выводы**

Данные не имеют пропусков, но имеется сильный дисбаланс классов, токсичных комментариев сильно меньше чем не токсичных, это может повлиять на результаты обучения. При подготовки данных для обучния необходимо выполнить стратификацию, чтобы сохранить соотношение классов в выборках.

## Подготовка данных к обучению на BERT

Разобьем данные на обучающую и тестовую выборку по соотношению 85:15

In [77]:
X_train, X_test, y_train, y_test = train_test_split(df.index.values,
                                                   df['toxic'].values,
                                                   test_size = 0.15,
                                                   random_state = RANDOM_STATE,
                                                   stratify = df['toxic'].values)

In [78]:
df['data_type'] = ['not_set'] * df.shape[0]
df.head()

,text,toxic,data_type
0,Explanation\nWhy the edits made under my usern...,0,not_set
1,D'aww! He matches this background colour I'm s...,0,not_set
2,"Hey man, I'm really not trying to edit war. It...",0,not_set
3,"""\nMore\nI can't make any real suggestions on ...",0,not_set
4,"You, sir, are my hero. Any chance you remember...",0,not_set


In [79]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_test, 'data_type'] = 'test'

In [80]:
df.groupby(['toxic', 'data_type']).count()

text
toxic data_type        
0     train      121640
      val         21466
1     train       13758
      val          2428

### Токенизация BERT

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case = True)

In [16]:
encoded_data_train = tokenizer.batch_encode_plus(df[df.data_type == 'train'].text.values,
                                                add_special_tokens = True,
                                                return_attention_mask = True,
                                                pad_to_max_length = True,
                                                max_length = 150,
                                                return_tensors = 'pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


c:\ProgramData\anaconda3\envs\ds_practicum_env\lib\site-packages\transformers\tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
encoded_data_test = tokenizer.batch_encode_plus(df[df.data_type == 'test'].text.values,
                                                return_attention_mask = True,
                                                pad_to_max_length = True,
                                                max_length = 150,
                                                return_tensors = 'pt')

In [19]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].toxic.values)

In [20]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']

labels_test = torch.tensor(df[df.data_type == 'test'].toxic.values)

Для обучения и тестирования нейронной сети подготовим датасет в формате тензора

In [24]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_test = TensorDataset(input_ids_test, 
                             attention_masks_test, 
                             labels_test)

In [25]:
print(len(dataset_train))
print(len(dataset_test))

135398
23894


### Обучение BERT

In [28]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = 2,
                                                      output_attentions = False,
                                                      output_hidden_states = False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
model.config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.38.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [30]:
dataloader_train = DataLoader(dataset_train,
                              sampler = RandomSampler(dataset_train),
                              batch_size = BATCH_SIZE)

dataloader_test = DataLoader(dataset_test,
                              sampler = RandomSampler(dataset_test),
                              batch_size = 32)

In [31]:
epochs = 10

optimizer = AdamW(model.parameters(),
                 lr = 1e-5,
                 eps = 1e-8)

c:\ProgramData\anaconda3\envs\ds_practicum_env\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [32]:
scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps = 0,
                                           num_training_steps = len(dataloader_train)*epochs)

In [33]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

<br/>    
<div class="alert alert-info">
<h2> Комментарий студента: <a class="tocSkip"> </h2>

<b>👋: </b>Привет! Подскажи пожалуйста правильно ли я понимаю что для того чтобы мне на двух GPU запустить обучение мне нужно этапы в функции ниже перевести на DataParallel или эту функцию надо на уровне модели применять?
</div> 

<br/> 

In [65]:
def evaluate(model, dataloader_val):

    # model = torch.nn.DataParallel(model)
    # model.cuda()
    model.to('cuda')
    
    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        #GPU
        batch = tuple(b.to(DEVICE) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]}
        
        with torch.no_grad():        
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [36]:
seed = RANDOM_STATE
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [37]:
for epoch in tqdm(range(1, epochs+1)):

    # model = torch.nn.DataParallel(model)
    # model.cuda()
    model.to(DEVICE)
    model.train()

    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(DEVICE) for b in batch)

        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()

        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    #f1 score
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Test (Validation) loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/33850 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.15293168258516596


  0%|          | 0/747 [00:00<?, ?it/s]

Validation loss: 0.12771632477292094
F1 Score (weighted): 0.9702840721892104


...

Validation loss: 0.312121919988347
F1 Score (weighted): 0.9701087388687304


Epoch 10:   0%|          | 0/33850 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.0025144718183257673


  0%|          | 0/747 [00:00<?, ?it/s]

Validation loss: 0.3357159744485248
F1 Score (weighted): 0.9697820057917811


### Сохранение и загрузка модели

In [62]:
# torch.save(model.to(DEVICE).state_dict(), 'C:\\Users\\Laba-ml\\LabaProjects\\BERT\\Model\\model.torch')

In [63]:
# model_load = BertForSequenceClassification.from_pretrained('bert-base-uncased',
#                                                       num_labels = 2,
#                                                       output_attentions = False,
#                                                       output_hidden_states = False).to(DEVICE)
# model_load.load_state_dict(torch.load('C:\\Users\\Laba-ml\\LabaProjects\\BERT\\Model\\model.torch'))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Прогнозирование и выводы

In [ ]:
_, predictions, true_test = evaluate(dataloader_test)

In [70]:
f1_score_func(predictions, true_test)

0.9697820057917811

Целью проекта было обучить модель для классификации комментариеа на позитивные и негативные. 

Заказчику было важно:

- значением метрики качества F1 >= 0.75

Выполненые задачи:
- Загрузка датасетов и предварительный обзор данных;
- Подготовка признаков и модели BERT;
- Обучение на тренировочном датасете;
- Проверка модели и анализ результатов на тестой выборке.

**Загрузка датасетов и подготовка признаков**
Данные не имеют пропусков, но имеется сильный дисбаланс классов, токсичных комментариев сильно меньше чем не токсичных, это может повлиять на результаты обучения. При подготовки данных для обучния выполнена стратификация, для сохранения соотношение классов в выборках.

Данные были разбиты по соотношению 85:15 на тестовую и обучающую выборку.


**Прогнозирование на тестовой выборки**

Итоговые результаты для тестовой выборки:

- `BERT`
    - F1 `0.969`

Условие по метрике выполнено

**Рекомендация:** Для наилучшего качества (если такое возможно), необходимо избавиться от дисбаланса классов, дополнить выборку токсичными комментариями